In [ ]:
def getMethodImpurity(numericalAttribute,value,examplesForCurAttr,method):
    #print("calculating Impurity")
    
    fra1 = train.loc[examplesForCurAttr,:]
    LessThanExamples = []
    MoreThanExamples = []
    for j in examplesForCurAttr:
        if  float(str(train.at[int(j),numericalAttribute])) <= value:
            LessThanExamples.append(int(j))
        else:
            MoreThanExamples.append(int(j))
    if len(LessThanExamples) ==0 or len(MoreThanExamples) == 0:    
        print("I am here and there is a problem")
    if len(LessThanExamples) ==0 or len(MoreThanExamples) == 0:
        return 999
    else:
        if getEntropyOfExamples(LessThanExamples) ==0 or getEntropyOfExamples(MoreThanExamples) == 0:
            print("yes i am here kartheek")        
        return ((len(LessThanExamples)*getEntropyOfExamples(LessThanExamples))+(len(MoreThanExamples)*getEntropyOfExamples(MoreThanExamples)))/len(examplesForCurAttr)
    


In [ ]:
def getEntropyOfExamples(examplesForCurAttr):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurAttr:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurAttr)
    PropOfNeg = NumberOfNegEx/len(examplesForCurAttr)
    if PropOfPos == 0:
        return (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))
    elif PropOfNeg == 0:
        return (-1*PropOfPos*(np.log(PropOfPos)/np.log(2)))
    else:
        return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

In [ ]:
class Node(object):
    def __init__(self,examples,decisionAttributeType,decisionAttribute,decisionAttributeValue,level):
        self.examples = examples
        self.decisionAttributeType = decisionAttributeType
        self.decisionAttribute = decisionAttribute        
        self.decisionAttributeValue = decisionAttributeValue
        self.level = level
        self.leftChild = None
        self.rightChild = None
        
    def __str__(self): 
        return "Decision attribute type is "+self.decisionAttributeType+"decisionAttribute is "+ self.decisionAttribute+" decisionAttributeValue is "+ str(self.decisionAttributeValue)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))

In [6]:
def BestOfAttributes(examplesForCurAttr,targetAttribute,method):
    entropyForCurrentExamples = getEntropyOfExamples(examplesForCurAttr)
    EntropyForCategoricalAttributes = {}
    for i in CategoricalAttributes:
        EntropyForCategoricalAttributes[i] = getImpurity(i,examplesForCurAttr,targetAttribute,method)
    BestOfCategoricalAttributes = min(EntropyForCategoricalAttributes, key=EntropyForCategoricalAttributes.get)
    BestOfCategoricalAttributesValue = min(list(EntropyForCategoricalAttributes.values()))
    dictForCategAttrUniqueValues = {}
    for i in NumericalAttributes:
        dictForNumAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValue = dictForNumAttrUniqueValues.copy()
    
    
    
    fra1 = train.loc[examplesForCurAttr,NumericalAttributes]
    #fra2 = fra1[:]
    #print(fra1)
    dictForNumAttrUniqueValues = {}
    for i in NumericalAttributes:
        dictForNumAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValue = dictForNumAttrUniqueValues.copy()
    print(dictForNumAttrUniqueValues)
    for i in NumericalAttributes:
        for j,val in enumerate(dictForNumAttrUniqueValues[i]):
            dictForEntropyUniqueValue[i][j] = getMethodImpurity(i,dictForNumAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForNumericalAttributes = {}
    for i in NumericalAttributes:
        EntropyValuesForNumericalAttributes[i]=min(dictForEntropyUniqueValue[i])
    BestOfNumericalAttributes = min(EntropyValuesForNumericalAttributes, key=EntropyValuesForNumericalAttributes.get)
    BestOfNumericalAttributesValue = min(EntropyValuesForNumericalAttributes.values())
    for i in dictForNumAttrUniqueValues[BestOfNumericalAttributes]:
        if i == BestOfNumericalAttributesValue:
            break
    BestSplitValue = i       
    if len(Attributes) > 5:
        if BestOfNumericalAttributesValue < BestOfCategoricalAttributesValue:
            return BestOfNumericalAttributes,BestSplitValue
        else: 
            return BestOfCategoricalAttributes,0
    else:
        return BestOfNumericalAttributes,BestSplitValue  



    
    
   


        
        
    

In [ ]:
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,level):
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:        
        r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        print(r)
        print("Type1")
        return r
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
        print(r)
        print("Type2")
        return r
    if getEntropyOfExamples(examplesForCurNode) < 0.01 or len(examplesForCurNode)<50:
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.5:
            r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
            numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            numberOfNodes  = numberOfNodes + 1
        print(r)
        print("Type3")
        return r
    
    BestAttribute,BestAttributeValue = BestOfAttributes(examplesForCurNode,targetAttribute,'misClasifi')
    
    if BestAttribute in NumericalAttributes:
        BestAttributeType = 'numerical'
    else:
        BestAttributeType = 'categorical'
    
    r = Node(examplesForCurNode,BestAttributeType,BestAttribute,BestAttributeValue,level)
    print(r)
    print("Type4")
    numberOfNodes  = numberOfNodes + 1
    examplesForLeftChildOfCurrentNode = []
    examplesForRightChildOfCurrentNode = []        
    if BestAttributeType == 'categorical':        
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(BestAttributeValue):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    else:
        for iter1 in examplesForCurNode:
            if  float(str(train.at[int(iter1),str(BestAttribute)])) <= float(str(BestAttributeValue)):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
        
    r.leftChild = DecisionTreeBuilder(examplesForLeftChildOfCurrentNode,targetAttribute,level+1)
    r.rightChild = DecisionTreeBuilder(examplesForRightChildOfCurrentNode,targetAttribute,level+1)
    
    return r

In [ ]:
def predict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decisionAttributeValue == 'Positive':
            return '1'
        else:
            #print("I am here in leaft")
            return '0'
    else:
        if r.decisionAttributeType == 'categorical': 
            
            AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])
            
            if AttrValue == str(r.decisionAttributeValue):
                return predict(i,r.leftChild,TestSet)
            else:
                return predict(i,r.rightChild,TestSet)
            
        elif r.decisionAttributeType == 'numerical':
            
            AttrValue = float(str(TestSet.at[int(i),str(r.decisionAttribute)]))
            
            if AttrValue <= float(str(r.decisionAttributeValue)):
                return predict(i,r.leftChild,TestSet)
            else:
                return predict(i,r.rightChild,TestSet)
            